# Traces 

## Analyse du fichier TRACES.TXT

Ce fichier est situé sur la machine 
 
C:\_SERVLOC\TRACES\DEV\TRACES.TXT

In [2]:
import pandas as pd
import os
import ipywidgets as widgets

fichier = "data/TRACE.txt"
fichier = "data/TRACE_INTEG2-R.txt"
fichier_sans_ext = os.path.splitext(os.path.basename(fichier))[0]

print("chargement de TRACES.txt")
%time traces = pd.read_csv(fichier,sep=";",header=None,names=["etat","jcl","classe","time"], parse_dates=["time"])


durée :0 days 01:44:01.420000


SelectionRangeSlider(index=(0, 0), options=(Timestamp('2019-02-21 14:59:48.820000'), Timestamp('2019-02-21 14:…

In [3]:
from IPython.display import display
debut = traces.min()['time']
fin = traces.max()['time']
duration =  fin - debut
print(f'durée :{duration}')

w = widgets.SelectionRangeSlider(
    options=traces['time']
    index=(0,traces)
)
display(w)

durée :0 days 01:44:01.420000


SelectionRangeSlider(index=(0, 0), options=(Timestamp('2019-02-21 14:59:48.820000'), Timestamp('2019-02-21 14:…

### Particularités du Fichier TRACES.TXT
C'est un fichier CSV séparateur ';'

Il est alimenté par les STEP *TRDEB* (DEBUT) et *TRFIN* (FIN) 

Si un JCL plante, alors le STEP *TRFIN* n'est pas executé, la ligne *FIN* correspondante n'existera pas

Si un JOB est lancé plusieurs fois il y aura autant de ligne *DEBUT* que de *TRDEB* executé 

## Liste des jobs 

In [29]:
jobs = traces['jcl'].unique()
jobs

Wall time: 4.98 ms


Wall time: 1.93 s


'file://....\\notebooks\\gantt_TRACE_INTEG2-R.html'

### Exemple de jcl avec 2 TRDEB et 1 TRFIN

`traces` est un pandas.DataFrame

Voici comment effectuer une requête simple :

`data` contient les lignes de `traces` avec un critère de selection sur la colonne `'jcl'` 

In [30]:
x = 'ORD_JCUSP100'

data = traces.loc[traces['jcl'] == x]
data


Wall time: 4.98 ms


Wall time: 1.93 s


'file://....\\notebooks\\gantt_TRACE_INTEG2-R.html'

In [31]:
from plotly import __version__
print (f'Version de plotly : {__version__}') 

Wall time: 4.98 ms


Wall time: 1.93 s


'file://....\\notebooks\\gantt_TRACE_INTEG2-R.html'

## Génération du diagramme de Gantt 


In [32]:
debut = traces.min()['time']
fin = traces.max()['time']
duration =  fin - debut
print(f'durée :{duration}')

Wall time: 4.98 ms


Wall time: 1.93 s


'file://....\\notebooks\\gantt_TRACE_INTEG2-R.html'

In [33]:

def jcl_as_dict(df, name) :
    '''
    recherche dans la DataFrame df les lignes correspondant au JCL nommé 'name'
    retourne un dictionaire avec le nom, la classe, les timestamps de début et de fin
    '''
    data = df.loc[df['jcl'] == name]
    start = data.min()['time']
    finish = data.max()['time']
    duration = finish-start
    return dict({'Task':name, 'classe': df.loc[data.index[0],'classe'], 'Start':start, 'Finish': finish, 'Duration':duration })

def gantt(traces, jobs):
    gantt_data = []
    for job in jobs: 
        infos = jcl_as_dict(traces, job)
        gantt_data.append(infos)
    return gantt_data


Wall time: 4.98 ms


Wall time: 1.93 s


'file://C:\\Users\\tlabbe\\PycharmProjects\\notebooks\\gantt_TRACE_INTEG2-R.html'


Consolider les données de `traces` dans `gantt_data`, triées sur la `classe`


In [34]:

%time gantt_data = sorted(gantt(traces,jobs), key=lambda k: k['classe'])
   


Wall time: 4.98 ms


Wall time: 1.93 s


'file://....\\notebooks\\gantt_TRACE_INTEG2-R.html'

Dessiner le diagramme de Gantt

In [35]:

import plotly.offline  as py
import plotly.figure_factory as ff
# help(ff.create_gantt)   


%time fig = ff.create_gantt(gantt_data,index_col='classe',task_names=['classes'],group_tasks=True, title=f'duration : {duration}', showgrid_x=True, height=800, width=1200)


%time py.plot(fig, filename=f'gantt_{fichier_sans_ext}.html')


Wall time: 4.98 ms


Wall time: 1.93 s


'file://....\\notebooks\\gantt_TRACE_INTEG2-R.html'

### Liste des classes et nombre de jobs lancés par classe :


In [36]:


liste = pd.DataFrame(data=gantt_data)

s = liste['classe'].value_counts()

x_axis = s.index.values
y_axis = s.values

import plotly.graph_objs as go
data = [go.Bar(x=x_axis.tolist(), y=y_axis.tolist())]
py.plot(data,filename=f'classes_repartition_{fichier_sans_ext}.html')    

'file://....\\notebooks\\classes_repartition_TRACE_INTEG2-R.html'

### Top 10 des Jobs les plus longs


In [37]:
# tri des job sur la durée décroissante


gantt_data = sorted(gantt(traces,jobs), key=lambda k: k['Duration'], reverse=True)

#recupperation des dix plus grandes valeurs
s=gantt_data[:10]

x = list(map(lambda d: {'jcl':d['Task'], 'Duration':d['Duration'] }, s))

tasks = list(map(lambda d:d['jcl'], x))
durations = list(map(lambda d:str(d['Duration']) , x))

#afficher une table des résultats
data = [go.Table(
    header=dict(values=['task','duration'],
                line = dict(color='#7D7F80'),
                fill = dict(color='#a1c3d1'),),
    cells=dict(values=[tasks,durations],
               line = dict(color='#7D7F80'),
               fill = dict(color='#EDFAFF'),
               align=['center','center']))
    ]
py.plot(data,filename=f'top_10_{fichier_sans_ext}.html')    


'file://....\\notebooks\\top_10_TRACE_INTEG2-R.html'

In [38]:
import numpy as np
df = pd.DataFrame(gantt_data)['Duration'] / np.timedelta64(1, 's')